In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


## Imports

In [22]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import warnings
warnings.filterwarnings('ignore')

## Set Device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyper Parameters

In [4]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
epochs = 1

## RNN network

In [31]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward prop
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

## Initialize the network

In [33]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

## Load the data

In [23]:
train_dataset = datasets.MNIST(root = 'dataset/', train=True, transform = transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root = 'dataset/', train=True, transform = transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)

## Loss and Optimizer

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train the network

In [39]:
for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):

        # Get the data to CUDA if possible
        data = data.to(device = device).squeeze(1)
        targets = targets.to(device = device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient Descent or Adam step
        optimizer.step()

## Check the accuracy on training and testig data

In [40]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on traning data")
    else:
        print("Checking accuracy on testing data")

    num_correct = 0
    num_samples = 0

    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(float(num_crrect)/float(num_samples)*100)

In [ ]:
model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)